In [ ]:
# Importando as Bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

In [ ]:
# Importando o DataFrame
df = pd.read_csv('https://www.football-data.co.uk/mmz4281/2223/E0.csv')

In [ ]:
# Pegando os Gols do Time Mandante
Home = df.loc[:,['HomeTeam','FTHG']]

# Pegando os Gols do Time Visitante
Away = df.loc[:,['AwayTeam','FTAG']]

In [ ]:
# Agrupando por time e somando a quantidade de gols
Home = Home.groupby('HomeTeam', as_index = False)['FTHG'].sum().sort_values(by='FTHG', ascending=True)
Away = Away.groupby('AwayTeam', as_index = False)['FTAG'].sum().sort_values(by='FTAG', ascending=True)

display(Home, Away)

In [ ]:
# Plotando o Número de Gols dos Mandantes
Home.plot(kind='bar', x ='HomeTeam', y='FTHG',alpha =0.6)
plt.style.use('fivethirtyeight')
plt.ylabel('Home Team Goals')
plt.xticks(rotation=90)
plt.legend('')
plt.xlabel('')
plt.title('Gols dos Mandantes por Times da Premier League 2022-2023',fontsize=15)
plt.legend('')
plt.rcParams.update({'font.size':10})

In [ ]:
# Plotando o Número de Gols dos Visitantes
Away.plot(kind= 'bar', x = 'AwayTeam', y='FTAG',alpha = 0.6)
plt.style.use('fivethirtyeight')
plt.ylabel('Away Team Goals')
plt.xticks(rotation=90)
plt.title('Gols dos Visitantes por Times da Premier League 2022-2023',fontsize=15)
plt.legend('')
plt.xlabel('')
plt.rcParams.update({'font.size':10})

In [ ]:
# Separando os  Resultados FT e HT 
Resultado = df.loc[:,['FTR','HTR']]

# Encontrar onde os resultados são no HT e FT são iguais
Mesmo_Resultado = Resultado[Resultado['FTR'] == Resultado['HTR']].reset_index().sort_index()

# Dropando a coluna de index do Mesmo_Resultado e Resetando o index
Mesmo_Resultado = Mesmo_Resultado.drop(columns = 'index')
Resultado.index += 1
Mesmo_Resultado.index += 1

In [ ]:
# Encontrando o último índice (o último jogo)
Mesmo_Resultado = Mesmo_Resultado.index.values[-1]
Mesmo_Resultado

In [ ]:
# Encontrando o último índice (o último jogo)
Resultado = Resultado.index.values[-1]
Resultado

In [ ]:
# Encontrar o número de jogos em que o resultado foi diferente
Resultado = Resultado - Mesmo_Resultado 
Resultado

In [ ]:
# Plotando o Gráfico de Pizza
fig = plt.figure(figsize =(10, 7)) 
plt.pie([Resultado, Mesmo_Resultado], explode = [0,0.1],autopct='%.2f%%', colors=['#7590a3','lightblue'],shadow = True) 
plt.style.use('fivethirtyeight')
plt.title('Porcentagem de Jogos que o Resultado mudou após o Intervalo',fontsize=15)
plt.axis('equal')
plt.legend(['HT ≠ FT', 'HT = FT'], loc='lower left')

In [ ]:
# Limpando os dados para encontrar o número de gols marcados 
Gols_Marcados_Home = Home.sort_values(by = 'HomeTeam', ascending=True)
Gols_Marcados_Away = Away.sort_values(by = 'AwayTeam', ascending=True)
Gols_Marcados = Gols_Marcados_Away.join(Gols_Marcados_Home)
Gols_Marcados['Total Goals Scored'] = Gols_Marcados.FTHG + Gols_Marcados.FTAG
Gols_Marcados = Gols_Marcados.drop(columns= ['HomeTeam'])
Gols_Marcados['Team']  = Gols_Marcados['AwayTeam']
Gols_Marcados = Gols_Marcados.drop(columns = 'AwayTeam')

In [ ]:
# Reordenando as colunas
Gols_Marcados = Gols_Marcados[['Team','FTHG','FTAG','Total Goals Scored']]
Gols_Marcados

In [ ]:
# Iniciando outro dataframe para encontrar o número de gols sofridos 
Gols_Sofridos = df.loc[:,['HomeTeam', 'AwayTeam',	'FTHG',	'FTAG']]

Gols_Sofridos_Home = Gols_Sofridos.groupby('HomeTeam',as_index = False)['FTAG'].sum().sort_values(by = 'FTAG', ascending=True)
Gols_Sofridos_Away = Gols_Sofridos.groupby('AwayTeam',as_index = False)['FTHG'].sum().sort_values(by = 'FTHG', ascending=True)

In [ ]:
# Encontrando o número de gols sofridos e limpando os dados
Gols_Sofridos = Gols_Sofridos_Away.join(Gols_Sofridos_Home)
Gols_Sofridos['Team'] = Gols_Sofridos['AwayTeam']
Gols_Sofridos = Gols_Sofridos.drop(columns= ['HomeTeam','AwayTeam'])
Gols_Sofridos['Total Goals Conceded'] = Gols_Sofridos.FTHG + Gols_Sofridos.FTAG
Gols_Sofridos = Gols_Sofridos[['Team','FTHG','FTAG','Total Goals Conceded']]
Gols_Sofridos

In [ ]:
# Calculando a Efetividade dos Times
Efetividade = Gols_Sofridos.join(Gols_Marcados['Total Goals Scored'])
Efetividade = Efetividade.reset_index()
Efetividade = Efetividade.drop(columns='index')
Efetividade = Efetividade.drop(columns= ['FTAG','FTHG'])
Efetividade['Goals Scored/Conceded Goals'] = Efetividade['Total Goals Scored'] / Efetividade['Total Goals Conceded']
Efetividade = Efetividade.sort_values(by='Goals Scored/Conceded Goals',ascending=True)
Efetividade

In [ ]:
# Criando um mapa de cores para o gráfico de dispersão
colors = cm.Blues(np.linspace(0,1,len(Efetividade)))

In [ ]:
# Plotando as equipes mais efetivas
ax = Efetividade.plot(x = 'Total Goals Conceded', y = 'Total Goals Scored', kind = 'scatter', s = 150, figsize = (15,10),color = colors)
# Plotando os rótulos para o gráfico de dispersão
Efetividade[['Total Goals Conceded','Total Goals Scored','Team']].apply(lambda x: ax.text(*x),axis=1)
# tornando os rótulos maiores
plt.rcParams.update({'font.size': 14})
plt.tick_params(axis='x', labelsize=14)
plt.tick_params(axis='y', labelsize=14)
# invertendo o eixo x para que o menor número de gols fique do lado direito em vez do esquerdo
plt.gca().invert_xaxis()
plt.xlabel('Total Goals Conceded', fontsize=18)
plt.ylabel('Total Goals Scored', fontsize=16)
cmap = cm.get_cmap('Spectral')
plt.title('Quem é o time mais eficaz da Premier League?', fontsize= 20)
plt.show()

In [ ]:
# Quanto a Bet365 acerta na sua precificação?
bet365 = df.loc[:,['B365H','B365A','FTR']]

In [ ]:
bet365['predicted result'] = 0
bet365['predicted result'] = bet365['B365H'] < bet365['B365A']
mask = bet365.applymap(type) != bool
d = {True: 'H', False: 'A'}
bet365 = bet365.where(mask, bet365.replace(d))
# Eliminando os resultados de empate
bet365 = bet365[bet365['FTR'] != 'D']
# Resetando o index
bet365 = bet365.reset_index()
bet365.index +=1
# Dropando a coluna do index
bet365 = bet365.drop(columns = 'index')
bet365

In [ ]:
# Encontrar onde o resultado previsto corresponde ao resultado real
Result_Ok = bet365[bet365['FTR'] == bet365['predicted result']]
Result_Ok = Result_Ok.reset_index()
Result_Ok = Result_Ok.drop(columns = ['index'])
Result_Ok.index += 1
Result_Ok

In [ ]:
# Encontrar o número de previsões em que a Bet365 acertou
Result_Ok = Result_Ok.index.values[-1]
Result_Ok

In [ ]:
# Encontrar o número de previsões em que a Bet365 errou
Result_No_Ok = bet365.index.values[-1] - Result_Ok
Result_No_Ok

In [ ]:
# Plotando o gráfico de pizza
fig = plt.figure(figsize =(10, 7)) 
plt.pie([Result_Ok, Result_No_Ok], explode = [0,0.1],autopct='%.2f%%', colors=['#77dd77','#ff6961'] ,shadow = True) 
plt.style.use('fivethirtyeight')
plt.title('% de vezes que a Bet365 previu corretamente a vitória do time da casa ou visitante',fontsize=15)
plt.axis('equal')
plt.legend(['Bet365 Acertou', 'Bet365 Errou'], loc='lower left')